In [133]:
import langchain
import langchain_community
import langchain_ollama

print("langchain version:", langchain.__version__)
print("langchain_community version:", langchain_community.__version__)
print("langchain_ollama version:", langchain_ollama.__version__)


langchain version: 0.3.25
langchain_community version: 0.3.24
langchain_ollama version: 0.3.3


In [2]:
from langchain_ollama import ChatOllama

In [4]:
def get_completion(prompt, model="llama3.1"):
    messages = [
        {"role": "system", "content": "You are a helpful assistant, while giving response to the user, do not output intro or extra detail"},
        {"role": "user", "content": prompt}
    ]
    
    llm = ChatOllama(
    model = "llama3.1",
    temperature = 0,
    )
    
    res = llm.invoke(messages)
    return res.content

In [5]:
get_completion("What is 1+1")

'2'

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [7]:
style = """American English \
in a calm and respectful tone
"""

In [8]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
response = get_completion(prompt)

In [10]:
response

"I'm really frustrated that my blender lid came loose and spilled my smoothie all over my kitchen walls. To make things even more inconvenient, the manufacturer's warranty doesn't cover the cost of cleaning up the mess. I could really use your assistance right now."

# Chat API: Ollama
## Model

In [39]:
model = ChatOllama(model = "llama3.1", temperature = 0)
model

ChatOllama(model='llama3.1', temperature=0.0)

# To reuse the prompt, let's save it in template

In [40]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
Instructions: {instructions}
"""

In [41]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [42]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['instructions', 'style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\nInstructions: {instructions}\n')

In [43]:
prompt_template.messages[0].prompt.input_variables

['instructions', 'style', 'text']

In [44]:
customer_style = """Americal English \
in a calm and respectful tone
"""

In [45]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [57]:
instructions = "While giving response to the user, do not output intro"

In [58]:
customer_messages = prompt_template.format_messages(
    style = customer_style,
    text = customer_email,
    instructions = instructions
)

In [59]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is Americal English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\nInstructions: While giving response to the user, do not output intro\n" additional_kwargs={} response_metadata={}


In [60]:
customer_response = model.invoke(customer_messages)

In [61]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and spilled smoothie all over my kitchen walls. Unfortunately, the warranty doesn't cover the cost of cleaning up the mess. I could use your help right now.


In [62]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [63]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [64]:
service_messages = prompt_template.format_messages(
    style = service_style_pirate,
    text = service_reply,
    instructions = instructions
)
print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```
Instructions: While giving response to the user, do not output intro



In [65]:
service_response = model.invoke(service_messages)
print(service_response.content)

Arrr, Yer lookin' fer a polite but firm warnin', eh? Alright then, matey!

"Dear valued customer, we regret to inform ye that our warranty don't cover cleanin' expenses fer yer kitchen. It seems ye had a wee mishap with yer blender, forgettin' the lid in the process o' startin' it up. We be sorry fer the inconvenience, but we cannae take responsibility fer misuse o' our fine appliances. Please be more careful in the future, savvy?"


# Output Parsers

In [66]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [67]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [74]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}

instructions: {instructions}
"""

In [88]:
instructions = "While giving response to the user, do not output intro or extra detail that's not required"

In [89]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['instructions', 'text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instructions', 'text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n\ninstructions: {instructions}\n'), additional_kwargs={})]


In [90]:
messages = prompt_template.format_messages(text=customer_review,
                                          instructions = instructions
                                        )

In [91]:
print(messages)

[HumanMessage(content="For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n\n\ninstructio

In [92]:
response = model.invoke(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


In [93]:
type(response.content)

str

In [94]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

# Parse the LLM output string into a Python dictionary

In [103]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [104]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [105]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [106]:
print(output_parser)

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')]


In [109]:
format_instructions = output_parser.get_format_instructions()

In [110]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [124]:
review_template_2 = """\
While giving response to user do not output intro or any extra details.
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [125]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review,
                                  format_instructions=format_instructions)

In [126]:
print(messages[0].content)

While giving response to user do not output intro or any extra details.
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the

In [127]:
response = model.invoke(messages)

In [128]:
print(response.content)

```json
{
    "gift": false,
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


In [129]:
output_dict = output_parser.parse(response.content)

In [130]:
output_dict

{'gift': False,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there"]}

In [131]:
type(output_dict)

dict

In [132]:
output_dict.get('delivery_days')

'2'